<a href="https://colab.research.google.com/github/simon-clematide/casdmit-fs21/blob/master/notebooks/zora_dewey_fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dewey-Klassifikation mit Zora-Material mit fasttext

> Indented block


Dieses Notebook demonstriert, wie einfach man ein gutes Klassifikations-Modell mit fastText trainieren kann.
Wir arbeiten mit der fasttext Python-Bibliothek.
Aus Effizienzgründen arbeiten wir hier mit einem kleineren Trainingsdatensatz.

## Das Python fasttext und spaCy Package installieren
Aktuellere Version hat [Bug](https://stackoverflow.com/questions/61787119/fasttext-0-9-2-why-is-recall-nan) in der label-spezifischen Evaluationsfunktion korrigiert 

In [ ]:
# ! pip install fasttext # schnell zu installieren, aber hat Bug bei test_label()
! pip install git+https://github.com/facebookresearch/fastText.git  # braucht mehr Zeit fürs Kompilieren

In [ ]:
! pip install spacy

In [ ]:
! python3 -m spacy download en_core_web_sm

In [ ]:
import logging
import spacy
nlp = spacy.load('en_core_web_sm')
nlp.disable_pipes("parser", "ner")

# Datenset: Zufällig ausgewählte Publikationen

In [ ]:
! curl https://files.ifi.uzh.ch/cl/siclemat/lehre/fs23/bibliosuisse/data/zora-eng-dewey.fasttext.tsv -o zora-eng-dewey.fasttext.tsv

### Format des Datensets 
 - Pro tabulator-separierte Zeile gibt es 2 Spalten
 - Spalte 1: [Dewey-Labels](https://en.wikipedia.org/wiki/List_of_Dewey_Decimal_classes)
 - Spalte 2: Titel und Abstract untokenisiert

In [ ]:
! head -n 10 zora-eng-dewey.fasttext.tsv

### Statistiken zum Datenset

In [ ]:
! wc -l zora-eng-dewey.fasttext.tsv

In [ ]:
!  cut -f 1 < zora-eng-dewey.fasttext.tsv | sort | uniq -c | sort -rn 

In [ ]:
def lemmatize_tsv(inputfile, outputfile, spacy_nlp, limit=999999):
    """Write tokenized and lemmatized version of data set"""

    with open(outputfile,"w",encoding="utf-8")as output:
        with open(inputfile,"r",encoding="utf-8") as input:
            for i,line in enumerate(input):
                labels, text = line.strip().split("\t")
                doc = nlp(text)
                print(labels,' '.join(token.lemma_ for token in doc).lower(),sep="\t",file=output)
                if i > limit:
                    break
                if i % 100 == 0:
                    print(f"Processed {i} records")


In [ ]:
# Download precomputed lemmatized data
#! curl https://files.ifi.uzh.ch/cl/siclemat/lehre/fs23/bibliosuisse/data/zora-eng-dewey.lemmatized.fasttext.tsv -o zora-eng-dewey.lemmatized.fasttext.tsv

In [ ]:
lemmatize_tsv("zora-eng-dewey.fasttext.tsv","zora-eng-dewey-10.lemmatized.fasttext.tsv",nlp,limit=10)

In [ ]:
! head zora-eng-dewey-10.lemmatized.fasttext.tsv

In [ ]:
lemmatize_tsv("zora-eng-dewey.fasttext.tsv","zora-eng-dewey.lemmatized.fasttext.tsv",nlp)

In [ ]:
! head zora-eng-dewey.lemmatized.fasttext.tsv

In [ ]:
def multilabel2singlelabel(inputfile, outputfile):
    """Reduce labels to the first label mentioned"""
    with open(outputfile,"w",encoding="utf-8")as output:
        with open(inputfile,"r",encoding="utf-8") as input:
            for i,line in enumerate(input):
                labels, text = line.strip().split("\t")
                label = labels.split(" ")[0]
                print(label, text, sep="\t",file=output)


In [ ]:
multilabel2singlelabel("zora-eng-dewey.lemmatized.fasttext.tsv","zora-eng-dewey.lemmatized.fasttext.single.tsv")

In [ ]:
! head zora-eng-dewey.lemmatized.fasttext.single.tsv

## Aufteilen der Daten in Trainings- und Testdaten
Erstellen von Training und Testdaten (Originaldaten sind zufällig geordnet)

In [ ]:
! head -n 9000 < zora-eng-dewey.lemmatized.fasttext.tsv > zora-eng-dewey.lemmatized.fasttext.train.tsv
! tail -n 1000 < zora-eng-dewey.lemmatized.fasttext.tsv > zora-eng-dewey.lemmatized.fasttext.test.tsv

In [ ]:
# optional erzeuge single label Daten
! head -n 9000 < zora-eng-dewey.lemmatized.fasttext.single.tsv > zora-eng-dewey.lemmatized.fasttext.train.tsv
! tail -n 1000 < zora-eng-dewey.lemmatized.fasttext.single.tsv > zora-eng-dewey.lemmatized.fasttext.test.tsv

In [ ]:
! echo TRAINING DATA STATISTICS
! cut -f 1 < zora-eng-dewey.lemmatized.fasttext.train.tsv | sort | uniq -c | sort -rn |head
! echo TEST DATA STATISTICS
! cut -f 1 < zora-eng-dewey.lemmatized.fasttext.test.tsv | sort | uniq -c | sort -rn |head

# Trainieren von Modell mit Python-Package
 - Dokumentation siehe https://fasttext.cc/docs/en/python-module.html

In [ ]:
import fasttext

[Word Embeddings](https://fasttext.cc/docs/en/pretrained-vectors.html) auf Wikipedia trainiert und wegen Speichergründen von mir auf 50 Dimensionen reduziert (Text-Format ist notwendig für supervisierte Klassifikation)

In [ ]:
! test -e wiki.en.50.vec || curl https://files.ifi.uzh.ch/cl/siclemat/lehre/fs23/bibliosuisse/data/wiki.en.50.vec -o wiki.en.50.vec

In [ ]:
# dauert ca. 40 Sekunden mit diesen Einstellungen
model = fasttext.train_supervised(
    input='zora-eng-dewey.lemmatized.fasttext.train.tsv', 
    pretrainedVectors="wiki.en.50.vec", # vortrainierte word embeddings
    epoch=10,  # Wie oft werden die Trainingsdaten benutzt
    minn=5,    # Minimal Subword-Länge in Buchstaben  
    maxn=5,    # Maximale Subword-Länge in Buchstaben 
    dim=50,    # Dimensionalität der Vektoren für die Repräsentation der Wörter und Subwords (muss gleich wie pretrainedVectors sein)
    lr=1,      # Learning Rate (Lernrate): Wie stark wird ein Fehler bestraft? 
    ws=10,
    verbose = True
    )

## Inspizieren des gelernten Modells

Welche Labels/Klassen kennt das Modell?

In [ ]:
print(model.labels)

Einen String klassifizieren und die Wahrscheinlichkeitsverteilung über allen möglichen Dewey erhalten:

In [ ]:
result = model.predict("interpersonal problems associate with multidimensional personality questionnaire traits in woman ",  
              k=5  # Gib die 5 besten Klassen aus
              )
for label,prob in zip(*result):
    print(label, round(prob,3))

Systematisches Testen des trainierten Models auf Testdaten:
 - k: Maximale Anzahl vorgeschlagener Labels
 - threshold: Minimale Wahrscheinlichkeit eine Labels, damit es als vorhergesagt gilt

In [ ]:
model.test("zora-eng-dewey.lemmatized.fasttext.test.tsv",k=3,threshold=0.25)

In [ ]:
def print_results(N, p, r):
    "Pretty print performance: N=Number of Samples, P/R@1=Precision/Recall of best prediction Acc=Accuracy "
    print(f"N\t{N}")
    print(f"P@k\t{p:.2f}")
    print(f"R@k\t{r:.2f}")
    print(f"Acc\t{r:.2f}")

In [ ]:
print_results(*model.test("zora-eng-dewey.lemmatized.fasttext.test.tsv",k=3,threshold=0.25))

Detaillierte Evaluation zu jedem einzelnen Label:
 - Precision: Anteil korrekter Klassifikationen einer Klasse
 - Recall: Anteil korrekt klassifizierter Elemente einer Klasse
 - f1score: Harmonisches Mittel von Precision und Recall

In [ ]:
data = model.test_label('zora-eng-dewey.lemmatized.fasttext.test.tsv',k=3, threshold=0.35)
sorted_data = sorted(data.items(), key=lambda x: x[1]['f1score'], reverse=True)
print(sorted_data)
for label, perf in sorted_data:
    print(label, perf)

## Vorhersagen und Wahrheit anzeigen

In [ ]:
!ls -lh

In [ ]:
test_data = []
with open("zora-eng-dewey.lemmatized.fasttext.test.tsv", mode="r",encoding="utf-8") as testfile:
    for line in testfile:
        test_data.append(line.strip().split("\t"))
test_data[:3]


In [ ]:
from collections import Counter
confusion_matrix = Counter()

# If given a list of strings, it will return a list of results as usually received for a single line of text.
predictions,probs = model.predict([text for _,text in test_data], k=3, threshold=0.25)

for i,preds in enumerate(predictions):
    labels = " ".join(sorted(preds)).replace('__label__','')
    if not labels:
        labels = '???'
    confusion_matrix[(test_data[i][0].replace('__label__',''),labels)] += 1

# korrekte 
print("CORRECT PREDICTIONS")
for (correct, predicted), count in confusion_matrix.most_common():
    if correct == predicted:
        print("TRUTH",correct, "SYSTEM",predicted, "COUNT",count)

# falsche 
print("\n\nWRONG PREDICTIONS")
for (correct, predicted), count in confusion_matrix.most_common():
    if correct != predicted:
        print("TRUTH",correct, "SYSTEM",predicted, "COUNT",count)

# Verbessern des Modells
Verbessern des Modells: Z.B. mehr Epochen, mehr Dimensionen, längere Buchstaben-N-Gramme, ...

Wichtigste Parameter:
```
   epoch N  # Beim Lernen wird das ganze Trainingsset N mal benutzt. Beeinflusst die Dauer des Trainings linear!
   dim N    # Länge der gelernten Vektoren für Wörter und Buchstaben-N-Gramme
   lr 0.N   # Initiale Lernrate: Bestimmt, wie stark die Vektoren verändert werden, wenn Fehler passieren. Während des Lernens wird die Lernrate immer kleiner.
   mmin N   # Minimale Länge der Subwords, d.h. Buchstaben-N-Gramme
   maxn N   # Maximale Länger der Subwords, d.h. Buchstaben-N-Gramme (falls N=0, werden keine Subwords benutzt, nur Wörter)
```

In [ ]:
model = fasttext.train_supervised(
    input='zora-eng-dewey.lemmatized.fasttext.train.tsv', 
    pretrainedVectors="wiki.en.50.vec", # vortrainierte word embeddings, können weggelassen werden
    epoch=20,  # Wie oft werden die Trainingsdaten benutzt
    minn=5,    # Minimal Subword-Länge in Buchstaben  
    maxn=5,    # Maximale Subword-Länge in Buchstaben 
    dim=50,    # Dimensionalität der Vektoren für die Repräsentation der Wörter und Subwords (muss gleich wie pretrainedVectors sein)
    lr=1,      # Learning Rate (Lernrate): Wie stark wird ein Fehler bestraft? 
    )
print_results(*model.test("zora-eng-dewey.lemmatized.fasttext.test.tsv"))
model.test_label('zora-eng-dewey.lemmatized.fasttext.test.tsv',k=3, threshold=0.25)

# Anhang: Embeddings

In [ ]:
! test -e wiki.en.50.bin || curl https://files.ifi.uzh.ch/cl/siclemat/lehre/fs23/bibliosuisse/data/wiki.en.50.bin -o wiki.en.50.bin

In [ ]:
full_model = fasttext.load_model('wiki.en.50.bin')

In [ ]:
full_model.get_nearest_neighbors('disease')

A is to B, like ? is to C model.get_analogies(A,B,C)

In [ ]:
full_model.get_analogies('man','woman','queen')

How to store the 400000 most frequent words in a smaller text format that is usable for supervised training.

In [ ]:
model=full_model
# Store only the 100,000 most frequent words
max_words = 400000
words = model.words[:max_words]
vectors = [model[word] for word in words]

# Save the subset of words and vectors to a text file
with open("model_subset.txt", "w", encoding="utf-8") as f:
    # Write the header with the vocabulary size and vector dimensionality
    f.write(f"{max_words} {model.get_dimension()}\n")

    # Write the vectors for each word
    for word, vector in zip(words, vectors):
        vector_str = " ".join([f"{x:.6f}" for x in vector])
        f.write(f"{word} {vector_str}\n")
